In [6]:
import random
import hashlib
import time

Ici on a la classe BlockPos qui permet d'initialiser un block PoS avec comme attributs : index, previoushash, data, validateur,timestamp, son hash.

la fonction calculatehash permet de calculer le hash de ce block en fonction des attributs cités précedemment.

la fonction validateblock permet de choisir le validateur de ce block de maniere aléatoire chaque participant a un nombre de coin misé et ce nombre de coin misé influence ce choix aléatoire, plus un participant a un nombre élevé de coin misé plus sa chance d'etre choisi comme validateur de ce block est grande.

In [ ]:
class BlockPoS:
    def __init__(self, index, previous_hash, data):
        self.index = index
        self.previous_hash = previous_hash
        self.data = data
        self.timestamp = time.time()
        self.validator = None
        self.current_hash = self.calculate_hash()

    # fonction pour calculer le hash du block
    def calculate_hash(self):
        block_content = f"{self.index}{self.previous_hash}{self.data}{self.timestamp}{self.validator}"
        return hashlib.sha256(block_content.encode()).hexdigest()

    # Nominer le "validator" qui va confirmer le block
    def validate_block(self, validators):
        total_stake = sum(stake for _, stake in validators)
        selection = random.uniform(0, total_stake)
        cumulative_stake = 0

        for validator, stake in validators:
            cumulative_stake += stake
            if cumulative_stake >= selection:
                self.validator = validator
                break
        # mise a jour du hash apres avoir assigner le validateur
        self.current_hash = self.calculate_hash()
        return self.validator  

On a ici la classe BlockchainPoS qui est l'ensemble des block avec comme attributs les validateurs de cette chaine de blocks, elle commence par le block appelé genesis block.

on a la fonction getlackblock pour prendre le block précedent.

on a la fonction add_block qui permet d'ajouter un nouveau block en appliquant les etapes citées précedemment.

la fonction printchain pour afficher nos blocks de la chaine avec les validateurs choisis pour chaque block les données et le hash.

In [8]:
class BlockchainPoS:
    def __init__(self):
        self.chain = [self.create_genesis_block()]
        self.validators = [("Validator1", 10), ("Validator2", 30), ("Validator3", 5)]

    def create_genesis_block(self):
        return BlockPoS(0, "0", "Genesis Block PoS")

    def get_last_block(self):
        return self.chain[-1]

    def add_block(self, new_block_data):
        previous_block = self.get_last_block()
        new_block = BlockPoS(len(self.chain), previous_block.current_hash, new_block_data)
        validator = new_block.validate_block(self.validators)
        self.chain.append(new_block)
        return validator

    def print_chain(self):
        for block in self.chain:
            print(f"Block {block.index}: [Validator: {block.validator}, Data: {block.data}, Hash: {block.current_hash}]")

Enfin on a ici la fonction qui nous permet de tester ces classes et  de calculer le temps d'execution de Proof of Stake.

In [9]:
# On teste le temps dexecution de Proof of Stake 
def test_pos():
    blockchain_pos = BlockchainPoS()
    start_time = time.time()
    validator = blockchain_pos.add_block("New Block PoS")
    end_time = time.time()
    validation_time = end_time - start_time
    blockchain_pos.print_chain()
    print(f"Validator: {validator}, Time taken for PoS validation: {validation_time:.6f} seconds")


In [10]:
test_pos()

Block 0: [Validator: None, Data: Genesis Block PoS, Hash: 300f2f1ec985055b812f22f0ae3f99297c4f5c1c0580c9453303f012d86d7c3f]
Block 1: [Validator: Validator2, Data: New Block PoS, Hash: de7e19b724ed4235515eed853a7060fdddf3746bc25eda392a0563b683e131a9]
Validator: Validator2, Time taken for PoS validation: 0.000000 seconds


En testant le Proof of Work et le Proof of Stake, on peut conclure que le Proof of Stake est généralement plus rapide avec un temps d'execution plus petit, et cela car il ne possède pas de concept de difficulté comme le Proof of Work(voir exercice2.ipynb atelier 1 github), cela dit plus la difficulté est élevé dans un Proof of Work plus le temps d'execution sera plus grand. Donc la comparaison des deux dépend généralement de la difficulté du Proof of Work. . Le nombre de validateurs dans le Proof of Stake peut aussi avoir un effet, mais il est généralement moins problématique pour les performances.